<a href="https://colab.research.google.com/github/harshy02/product-data-scraping/blob/main/Harshita_Mittal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PART 1

In [ ]:
!pip install beautifulsoup4
!pip install requests

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time

def scrape_amazon_products(url, num_pages=20):
    all_products = []
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    }

    for page in range(1, num_pages + 1):
        page_url = f"{url}&page={page}"
        response = requests.get(page_url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            products = soup.find_all('div', {'data-asin': True})

            # iterating over extracted data
            for product in products:
                product_data = {}

            # fetching desired data
                try:
                    product_name = product.find('span', {'class': 'a-size-medium'}).text.strip()
                    product_data['Product Name'] = product_name

                    # error handling
                except:
                    product_data['Product Name'] = 'N/A'

                try:
                    product_url = 'https://www.amazon.in' + product.find('a', {'class': 'a-link-normal'})['href']
                    product_data['Product URL'] = product_url
                except:
                    product_data['Product URL'] = 'N/A'

                try:
                    product_price = product.find('span', {'class': 'a-offscreen'}).text.strip()
                    product_data['Product Price'] = product_price
                except:
                    product_data['Product Price'] = 'N/A'

                try:
                    product_rating = float(product.find('span', {'class': 'a-icon-alt'}).text.split()[0])
                    product_data['Rating'] = product_rating
                except:
                    product_data['Rating'] = 'N/A'

                try:
                    product_reviews = int(product.find('span', {'class': 'a-size-base'}).text.replace(',', ''))
                    product_data['Number of Reviews'] = product_reviews
                except:
                    product_data['Number of Reviews'] = 'N/A'

                all_products.append(product_data)

        time.sleep(2)  # delay after each page

    return all_products

In [ ]:
# calling function
amazon_url = "https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1"
scraped_products = scrape_amazon_products(amazon_url, num_pages=20)

In [ ]:
with open('amazon_products_part1.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Product Name', 'Product URL', 'Product Price', 'Rating', 'Number of Reviews']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(scraped_products)

In [ ]:
from google.colab import files
files.download('amazon_products_part1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PART 2

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time

def scrape_amazon_products(url, num_pages=20, num_products=200):
    all_products = []
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    }
    products_scraped = 0

    for page in range(1, num_pages + 1):
        page_url = f"{url}&page={page}"
        #fetch page content
        response = requests.get(page_url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            products = soup.find_all('div', {'data-asin': True})

            for product in products:
                if products_scraped >= num_products:
                    break
                product_data = {}

                try:
                    product_name = product.find('span', {'class': 'a-size-medium'}).text.strip()
                    product_data['Product Name'] = product_name
                except:
                    product_data['Product Name'] = 'N/A'

                try:
                    product_url = 'https://www.amazon.in' + product.find('a', {'class': 'a-link-normal'})['href']
                    product_data['Product URL'] = product_url

                    # Fetch additional information from the product page
                    product_response = requests.get(product_url, headers=headers)
                    if product_response.status_code == 200:
                        product_soup = BeautifulSoup(product_response.content, 'html.parser')

                        # Description
                        try:
                            product_description = product_soup.find('meta', {'name': 'description'})['content']
                            product_data['Description'] = product_description
                        except:
                            product_data['Description'] = 'N/A'

                        # ASIN
                        try:
                            asin = product_soup.find('th', text='ASIN').find_next_sibling('td').text.strip()
                            product_data['ASIN'] = asin
                        except:
                            product_data['ASIN'] = 'N/A'

                        # Product Description
                        try:
                            long_description = product_soup.find('div', {'id': 'productDescription'}).text.strip()
                            product_data['Product Description'] = long_description
                        except:
                            product_data['Product Description'] = 'N/A'

                        # Manufacturer
                        try:
                            manufacturer = product_soup.find('a', {'id': 'bylineInfo'}).text.strip()
                            product_data['Manufacturer'] = manufacturer
                        except:
                            product_data['Manufacturer'] = 'N/A'

                except:
                    product_data['Product URL'] = 'N/A'

                # store product data
                all_products.append(product_data)
                products_scraped += 1

        if products_scraped >= num_products:
            break

        time.sleep(2)

    return all_products

In [ ]:
amazon_url = "https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1"
scraped_products = scrape_amazon_products(amazon_url, num_pages=20)

In [ ]:
with open('amazon_products_part2.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Product Name', 'Product URL', 'Product Price', 'Rating', 'Number of Reviews', 'Description', 'ASIN', 'Product Description','Manufacturer' ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(scraped_products)

In [ ]:
from google.colab import files
files.download('amazon_products_part2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>